<a target="_blank" href="https://colab.research.google.com/github/lukebarousse/Int_SQL_Data_Analytics_Course/blob/main/7_Basic_Query_Optimization/1_Explain.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Explain

## Overview

### 🥅 Analysis Goals

Analyze customer-level revenue, order behavior, and cohort classification to assess purchasing patterns and customer value.  

- **Plan Query Execution:** Use `EXPLAIN` to understand how PostgreSQL will execute the query, identifying potential inefficiencies like sequential scans or costly joins.  
- **Measure Actual Query Performance:** Use `EXPLAIN ANALYZE` to execute the query while collecting real performance metrics, comparing estimated vs. actual execution times for optimization.

### 📘 Concepts Covered

- `EXPLAIN`
- `EXPLAIN ANALYZE`

---
## EXPLAIN

### 📝 Notes

**`EXPLAIN`**  

- **EXPLAIN**: Displays the execution plan of a SQL query, showing how PostgreSQL will execute it.

- Syntax:  
  ```sql
  EXPLAIN SELECT column FROM table WHERE condition;
  ```
  
- **`EXPLAIN ANALYZE`**: Executes the query and provides actual execution times, row estimates, and other runtime details.
  ```sql
  EXPLAIN ANALYZE SELECT column FROM table WHERE condition;
  ```

- Helps with query optimization by showing:
  - Index usage
  - Join methods (`Nested Loop`, `Hash Join`, `Merge Join`)
  - Sequential vs. index scans
  - Estimated vs. actual row counts

- **Example Output** (simplified):
  ```
  Seq Scan on users  (cost=0.00..18.50 rows=850 width=64)
  ```
  - `Seq Scan`: PostgreSQL is doing a sequential scan (no index used).
  - `cost`: Estimated startup and total cost.
  - `rows`: Estimated number of rows.
  - `width`: Estimated row size in bytes.

- **Use Cases**:
  - Debugging slow queries
  - Checking if indexes are being used
  - Understanding query performance bottlenecks

### 💻 Final Result

- Understand how PostgreSQL plans to execute the query without running it, helping identify potential inefficiencies like sequential scans or costly joins.
- Execute the query while collecting actual performance metrics, allowing comparison between estimated and real execution times to optimize query performance.

### 💡 Note

For the queries below since we've already done them the explanation is explaining the results of the `EXPLAIN` plan.

#### Basic Execution Plan

**`EXPLAIN`**

1. Using the last query from `String Formatting` chapter, add `EXPLAIN` to the beginning of the query. 
    - **Hash Left Join:** PostgreSQL joins `customer` (104,990 rows) and `sales_data` (37,024 rows).  
    - **Sequential Scan on `customer`:** Scans all rows instead of using an index.  
    - **Hash Aggregation on `sales`:** Groups `sales` (199,873 rows) by `customerkey`, calculating revenue, orders, and cohort year.  
    - **Sequential Scan on `sales`:** No filtering, so the entire table is scanned.  

In [ ]:
EXPLAIN
WITH sales_data AS (
    SELECT
        customerkey,
        EXTRACT(YEAR FROM MIN(orderdate)) AS cohort_year,
        SUM(quantity * netprice * exchangerate) AS net_revenue,
        COUNT(orderkey) AS num_orders
    FROM sales
    GROUP BY customerkey
)
SELECT
    c.customerkey,
    s.cohort_year,
    CONCAT(TRIM(c.givenname), ' ', TRIM(c.surname)) AS cleaned_name,
    COALESCE(s.net_revenue, 0) AS net_revenue,
    COALESCE(s.num_orders, 0) AS total_orders,
    s.net_revenue / NULLIF(s.num_orders, 0) AS avg_order_value
FROM customer c
LEFT JOIN sales_data s ON c.customerkey = s.customerkey;


<img src="../Resources/query_results/7_explain_1.png" alt="Query Results 1" style="width: 90%; height: auto;">

2. To view the `EXPLAIN` plan visually go to the `E` option and click it.
    - **Sequential Scan on `sales`** – Reads all rows without an index, leading to a full table scan.  
    - **Aggregation on `sales`** – Groups data by `customerkey` to calculate cohort year, net revenue, and order count.  
    - **SubQuery Scan (Materialized CTE)** – Stores aggregated results in a temporary set instead of inlining.  
    - **Hash on `sales_data`** – Builds a hash table for efficient joining.  
    - **Sequential Scan on `customer`** – Reads all rows without an index.  
    - **Hash Left Join (`customer` → `sales_data`)** – Joins `customer` with hashed `sales_data` for efficiency.  

<img src="../Resources/query_results/7_explain_3.gif" alt="Query Results 1" style="width: 90%; height: auto;">

<img src="../Resources/query_results/7_explain_3.png" alt="Query Results 1" style="width: 90%; height: auto;">

#### Full Execution Plan

**`EXPLAIN ANALYZE`**

1. Using `EXPLAIN ANALYZE` to the beginning of the query. 
    - **Total Execution Time:** **101.254 ms.**  
    - **Hash Join Execution:** **60.479 to 99.183 ms**, most of the query cost is in hashing `sales_data`.  
    - **Subquery Scan on `sales_data`:** **50.604 to 57.226 ms**, shows the CTE is materialized.  
    - **Sequential Scan on `sales`:** **0.008 to 10.471 ms**, confirms full table scan.  

In [ ]:
EXPLAIN ANALYZE
WITH sales_data AS (
    SELECT
        customerkey,
        EXTRACT(YEAR FROM MIN(orderdate)) AS cohort_year,
        SUM(quantity * netprice * exchangerate) AS net_revenue,
        COUNT(orderkey) AS num_orders
    FROM sales
    GROUP BY customerkey
)
SELECT
    c.customerkey,
    s.cohort_year,
    CONCAT(TRIM(c.givenname), ' ', TRIM(c.surname)) AS cleaned_name,
    COALESCE(s.net_revenue, 0) AS net_revenue,
    COALESCE(s.num_orders, 0) AS total_orders,
    s.net_revenue / NULLIF(s.num_orders, 0) AS avg_order_value
FROM customer c
LEFT JOIN sales_data s ON c.customerkey = s.customerkey;


<img src="../Resources/query_results/7_explain_2.png" alt="Query Results 1" style="width: 90%; height: auto;">